# 函数调用(Function Calling)

文心一言提供函数调用功能，模型根据用户需求以及对函数的描述确定何时以及如何调用函数。函数调用功能的典型使用流程如下：

(1) 用户提供对一组函数的名称、功能、请求参数（输入参数）和响应参数（返回值）的描述； 

(2) 模型根据用户需求以及函数描述信息，智能确定是否应该调用函数、调用哪一个函数、以及在调用该函数时需要如何设置输入参数； 

(3) 用户根据模型的提示调用函数，并将函数的响应传递给模型； 

(4) 模型综合对话上下文信息，以自然语言形式给出满足用户需求的回答。

借由函数调用，用户可以从大模型获取结构化数据，进而利用编程手段将大模型与已有的内外部API结合以构建应用。

在ERNIE Bot中，erniebot.ChatCompletion.create接口提供函数调用功能。关于该接口的更多详情请参考[ChatCompletion API文档](../../docs/api_reference/chat_completion.md)。

<div align="left">

| 字段名称    | 数据类型 | 字段描述                         |
| :-------------- | ---------- | ---------------------------------- |
| name        | str      | API（函数）名称                  |
| description | str      | API(函数）功能描述               |
| parameters  | dict     | API(函数）输入参数（以dict输入） |
| responses   | dict     | API(函数）输出结果（以dict返回） |

</div>

parameter 和 responses 的相关字段情况如下


| 字段名     | 数据类型  | 描述                                                |
| :------------- | ----------- | ----------------------------------------------------- |
| type       | str       | 此层级一般为object                                  |
| properties | dict      | 变量的详细情况，键为变量名，值为变量的相关信息（dict），该字典详见下表                 |
| required   | List[str] | 函数的输入参数中必须含有的变量，responses不含此字段 |


对于properties中的变量的字典，相关的键值对如下


| properties中的变量字典的元素 | 是否必须     | 描述                 |
| :----------------------------- | ---------- | ---------------------- |
| type                         | Required | 此层级一般为string，也可以使用integral等，但是因为整体为文本返回，差别不是特别大，使用时请注意函数内部转换     |
| description                  | Optional   | 该变量的描述         |
| enum                         | Optional   | 该变量的可选参数 |
| default                         | Optional   | 该变量的默认参数 |




以下是一个简单的使用示例

## Step 1 
用户提供对一组函数的名称、功能、请求参数（输入参数）和响应参数（返回值）的描述（可使用以下类进行统一管理）。 

In [1]:
functions = [
    {
        "name": "add_function",
        "description": "计算两个数的和",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "string", "description": "第一个数"},
                "b": {"type": "string", "description": "第二个数"},
            },
            "required": ["a", "b"],
        },
        "responses": {"type": "object", "properties": {}},
    },
    {
        "name": "minus_function",
        "description": "计算两个数的差",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "string", "description": "第一个数"},
                "b": {"type": "string", "default": "1.0"},
            },
            "required": ["a"],
        },
        "responses": {"type": "object", "properties": {}},
    },
]

## Step 2
模型根据用户需求以及函数描述信息，智能确定是否应该调用函数、调用哪一个函数、以及在调用该函数时需要如何设置输入参数； 


在有了添加完function之后，即可调用function calling功能让大模型进行思考以及决策
function call功能中返回的response的相关字段为

| name     | thoughts     | arguments  |
| :----------- | -------------- | ------------ |
| 函数名称 | 模型思考过程 | 函数的输入 |


In [14]:
import erniebot
from typing import List

erniebot.api_type = "aistudio"
erniebot.access_token = "<eb-access-token>"

response = erniebot.ChatCompletion.create(
    model="ernie-4.0",
    messages=[
        {
            "role": "user",
            "content": "1加2等于多少",
        },
    ],
    functions=functions,
)
function_call = response.function_call
print(function_call)

{'name': 'add_function', 'thoughts': '用户的问题是关于1加2的计算，我可以使用add_function工具来计算这两个数的和。', 'arguments': '{"a":"1","b":"2"}'}


## Step 3
用户根据模型的提示调用函数，并将函数的响应传递给模型； 

In [15]:
function_return = eval(response.get_result()["name"])(**eval(response.get_result()["arguments"]))
function_return

3.0

# Step 4[Option]
模型综合对话上下文信息，以自然语言形式给出满足用户需求的回答。

这一步不是必须的，用户可以不经过此步骤直接输出答案，这一步主要针对有需要再使用大模型进行进一步提炼、总结插件返回结果的场景，比如搜索插件、代码执行插件、计算器插件等别的插件。

In [16]:
chat_messages = [
    {
        "role": "user",
        "content": "1加2等于多少",
    },
    {
        "role": "assistant",
        "content": None,
        "function_call": function_call,
    },
    {
        "role": "function",
        "name": function_call["name"],
        "content": function_return,
    },
]

response = erniebot.ChatCompletion.create(
    model="ernie-4.0",
    messages=chat_messages,
    functions=functions,
)
print(response.get_result())

1加2的和是3.0。如果您还有其他问题或需要我进行其他计算，请随时告诉我。


## Final
最终整体展示以及调用（整合了Step2-Step4），可以通过以下函数进行整体的串型调用执行，获取最终想要的结果。

In [17]:
import time


def function_call(
    query: str,
    functions: List[str],
) -> str:
    messages = [
        {
            "role": "user",
            "content": query,
        }
    ]
    # Step 2: 模型选择使用哪个函数/API
    response = erniebot.ChatCompletion.create(
        model="ernie-4.0",
        messages=messages,
        functions=functions,
    )

    assert hasattr(response, "function_call")
    function_call = response.function_call

    assert hasattr(response.get_result(), "thoughts")
    print("[Thought]:", response.get_result()["thoughts"])

    messages.append(
        {
            "role": "assistant",
            "content": None,
            "function_call": function_call,
        }
    )
    # Step 3: 执行函数调用
    print("[函数调用]结果计算中...")
    function_return = eval(response.get_result()["name"])(**eval(response.get_result()["arguments"]))

    messages.append(
        {
            "role": "function",
            "name": function_call["name"],
            "content": function_return,
        }
    )

    time.sleep(1)  # 防止频繁访问报错

    # Step 4: 回传模型进行修饰[Option]
    print("[回传]最终结果生成中...")
    response = erniebot.ChatCompletion.create(
        model="ernie-4.0",
        messages=messages,
        functions=functions,
    )
    print("[最终回答]", response.result)
    return response.result


function_call("512加798等于多少", functions=functions)

[Thought]: 用户想要知道512和798的和。我可以使用add_function工具来计算这两个数的和。
[函数调用]结果计算中...
[回传]最终结果生成中...
[最终回答] 512加798的结果是1310。如果你还有其他问题，随时告诉我。


'512加798的结果是1310。如果你还有其他问题，随时告诉我。'